## Data processing

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

In [2]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [3]:
df=pd.read_pickle('/home/centos/data/typed_all_v1_2.pkl')
del df['licenseInfo.drmSystem']

### remove data 'timestamp' == 0

In [4]:
df = df[df['timestamp']!=0]

In [5]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.index = df['timestamp']
del df['timestamp']

In [6]:
df = df.sort_index()

### error code 종류와 빈도수 

In [7]:
df['errorCode'].value_counts()

 0.000000e+00    17861
-1.007020e+05      440
-3.800000e+01      238
-2.005000e+03       38
-1.004000e+03       37
-1.002000e+03       22
-1.008700e+05       14
-1.008740e+05        7
-1.008590e+05        3
-1.200000e+01        1
-1.014160e+05        1
-2.147484e+09        1
Name: errorCode, dtype: int64

### errorcode==0,networkErrors == 0 , mnc == 10 제거 -> 시간별 최빈값으로 resampling하려고할때 이 값들이 모두 대표값이 되어버림

In [8]:
df_errors = df[df['errorCode']!=0.0]
df_networkErrors = df[df['networkErrors']!=0.0]
df_mnc = df[df['networkInfo.carrier.mnc']!= '10']

In [9]:
df_mnc['networkInfo.carrier.mnc']

timestamp
2018-11-06 02:14:03    51
2018-11-06 02:15:04    50
2018-11-06 02:19:40    51
2018-11-06 02:19:54    51
2018-11-06 02:20:25    51
2018-11-06 02:20:35    51
2018-11-06 02:20:39    51
2018-11-06 02:21:19    51
2018-11-06 02:24:28    51
2018-11-06 02:24:45    51
2018-11-06 02:25:46    50
2018-11-06 02:28:25    51
2018-11-06 02:29:36    20
2018-11-06 02:30:01    51
2018-11-06 02:30:46    51
2018-11-06 02:31:17    20
2018-11-06 02:31:59     0
2018-11-06 02:35:30    51
2018-11-06 02:36:12    51
2018-11-06 02:36:38    51
2018-11-06 02:38:20    51
2018-11-06 02:41:24    51
2018-11-06 02:43:20    51
2018-11-06 02:49:02    51
2018-11-06 02:49:21    20
2018-11-06 02:50:17    20
2018-11-06 02:50:20    20
2018-11-06 02:50:27    20
2018-11-06 02:51:10    51
2018-11-06 02:52:20    20
                       ..
2018-11-07 23:21:46    20
2018-11-07 23:23:28    20
2018-11-07 23:27:26    51
2018-11-07 23:28:06    51
2018-11-07 23:29:47    51
2018-11-07 23:30:01    20
2018-11-07 23:34:03    51
20

### 10분 단위로 grouping 하고 그 시간의 mean값을 대표값으로 저장(numeric value)

In [10]:
numeric_mean_value=df.groupby(pd.Grouper(freq='10T') ).mean()

### 10분 단위로 grouping 하고 그 시간의 최빈값을 대표값으로 저장( categorical value)

In [11]:
top_errorCode=df_errors['errorCode'].groupby(pd.Grouper(freq='10T') ).describe().top
top_networkErrors=df_networkErrors['networkErrors'].groupby(pd.Grouper(freq='10T') ).describe().top
top_mnc=df_mnc['networkInfo.carrier.mnc'].groupby(pd.Grouper(freq='10T') ).describe().top
top_networkType=df['networkInfo.type'].groupby(pd.Grouper(freq='10T') ).describe().top
top_contentType=df['content_type'].groupby(pd.Grouper(freq='10T') ).describe().top
top_device=df['device'].groupby(pd.Grouper(freq='10T') ).describe().top


### 10분치 데이터 대표값으로 resampling

In [12]:
result=pd.concat([numeric_mean_value, top_contentType, top_device, top_errorCode, top_networkErrors, top_mnc, top_networkType], axis=1, sort=False)




In [13]:
result.columns = ['estimatedBandwidth', 'fragmentSum.bitrate', 'fragmentSum.downloadTime',
       'fragmentSum.duration', 'fragmentSum.fragmentIndex', 'fragmentSum.size',
       'frameDropped', 'licenseInfo.elapsedTime', 'maxDecodingTime',
       'position', 'qualityChangedCount', 'bufferingTime', 'avgDecodingTime',
       'bufferedDuration', 'currentPosition', 'top_contentType', 'top_device', 'top_errorCode', 'top_networkErrors', 'top_mnc', 'top_networkType']

### 10분동안 가장 많이 발생한 한 값(e.g. errorCode == 0) 

In [14]:
result.top_errorCode = result.top_errorCode.fillna(0)
result.top_networkErrors=result.top_networkErrors.fillna(0)

In [15]:
result.dtypes

estimatedBandwidth           float64
fragmentSum.bitrate          float64
fragmentSum.downloadTime     float64
fragmentSum.duration         float64
fragmentSum.fragmentIndex    float64
fragmentSum.size             float64
frameDropped                 float64
licenseInfo.elapsedTime      float64
maxDecodingTime              float64
position                     float64
qualityChangedCount          float64
bufferingTime                float64
avgDecodingTime              float64
bufferedDuration             float64
currentPosition              float64
top_contentType                int64
top_device                     int64
top_errorCode                float64
top_networkErrors            float64
top_mnc                       object
top_networkType                int64
dtype: object

In [16]:
result.top_contentType = result.top_contentType.astype('float64')
result.top_device = result.top_device.astype('float64')
result.top_networkType = result.top_networkType.astype('float64')
result.top_mnc = result.top_mnc.astype('float64')

In [17]:
result.to_pickle('dataframe_10T.pkl')

### x, y data 할당

In [18]:
y_data=result.top_errorCode.values
del result['top_errorCode']
x_data = result.values

In [19]:
x_data.shape

(276, 20)

In [20]:
y_data = y_data.reshape(276,1)

In [21]:
result.dtypes

estimatedBandwidth           float64
fragmentSum.bitrate          float64
fragmentSum.downloadTime     float64
fragmentSum.duration         float64
fragmentSum.fragmentIndex    float64
fragmentSum.size             float64
frameDropped                 float64
licenseInfo.elapsedTime      float64
maxDecodingTime              float64
position                     float64
qualityChangedCount          float64
bufferingTime                float64
avgDecodingTime              float64
bufferedDuration             float64
currentPosition              float64
top_contentType              float64
top_device                   float64
top_networkErrors            float64
top_mnc                      float64
top_networkType              float64
dtype: object

In [22]:
result = result.interpolate()
result.fillna(0)

,estimatedBandwidth,fragmentSum.bitrate,fragmentSum.downloadTime,fragmentSum.duration,fragmentSum.fragmentIndex,fragmentSum.size,frameDropped,licenseInfo.elapsedTime,maxDecodingTime,position,qualityChangedCount,bufferingTime,avgDecodingTime,bufferedDuration,currentPosition,top_contentType,top_device,top_networkErrors,top_mnc,top_networkType
timestamp,,,,,,,,,,,,,,,,,,,,
2018-11-06 02:10:00,8.892626e+07,1.422250e+07,2134.068966,30372.344828,15.241379,3.147544e+06,0.034483,975.758621,6.172414,1.957291e+06,0.517241,160.586207,1.137931,312.724138,1.957291e+06,1.0,10.0,10.0,51.0,2.0
2018-11-06 02:20:00,7.350672e+07,1.738292e+07,3251.234043,35432.787234,20.468085,3.316558e+06,0.021277,1003.276596,6.702128,1.295265e+06,0.723404,197.574468,1.446809,438.744681,1.295264e+06,1.0,10.0,4.0,51.0,2.0
2018-11-06 02:30:00,6.497422e+07,1.437704e+07,2860.760870,24133.608696,14.978261,3.236099e+06,0.021739,961.521739,8.608696,1.622858e+06,0.630435,104.369565,2.043478,84.413043,1.622858e+06,1.0,10.0,13.0,51.0,2.0
2018-11-06 02:40:00,9.335001e+07,6.679514e+06,1672.850000,16440.750000,12.900000,1.260177e+06,0.000000,928.975000,7.750000,1.319017e+06,0.800000,144.025000,2.025000,278.125000,1.319017e+06,1.0,10.0,0.0,51.0,2.0
2018-11-06 02:50:00,8.437353e+07,1.364711e+07,2480.754386,27726.543860,15.263158,3.052809e+06,0.052632,1045.070175,8.000000,1.475737e+06,0.771930,168.859649,1.578947,545.771930,1.475737e+06,1.0,10.0,1.0,51.0,2.0
2018-11-06 03:00:00,7.149909e+07,9.916105e+06,2319.931507,24891.890411,19.849315,2.231573e+06,0.013699,975.808219,7.835616,1.138651e+06,0.821918,173.547945,1.506849,168.082192,1.138651e+06,1.0,10.0,3.0,20.0,2.0
2018-11-06 03:10:00,6.396485e+07,1.128921e+07,2656.224138,22610.810345,15.120690,2.169355e+06,0.034483,1024.034483,7.948276,1.923919e+06,0.775862,457.758621,2.086207,158.344828,1.923919e+06,1.0,10.0,2.0,20.0,2.0
2018-11-06 03:20:00,1.265294e+08,1.063980e+07,2870.777778,23991.209877,21.814815,2.189145e+06,0.024691,1005.975309,8.123457,1.204875e+06,0.938272,345.716049,2.160494,286.654321,1.204875e+06,1.0,10.0,2.0,51.0,2.0
2018-11-06 03:30:00,5.865969e+07,9.839586e+06,2766.865854,25104.292683,17.121951,2.186388e+06,0.024390,1082.207317,6.634146,1.239439e+06,0.914634,145.256098,1.329268,114.634146,1.239439e+06,1.0,10.0,0.0,51.0,2.0


In [23]:
from sklearn import preprocessing # Min-Max Standardzation

min_max_scaler = preprocessing.MinMaxScaler()
x_data = min_max_scaler.fit_transform(x_data)


training_idx = np.random.randint(y_data.shape[0], size=int(y_data.shape[0] * 0.8))
test_idx = np.random.randint(y_data.shape[0], size=int(y_data.shape[0] * 0.2))

x_train, x_test = x_data[training_idx,:], x_data[test_idx,:]


In [24]:
y_train, y_test = y_data[training_idx,:], y_data[test_idx,:]

x_train.shape, x_test.shape

((220, 20), (55, 20))

In [25]:
from sklearn import tree
from sklearn import metrics
result_tree = tree.DecisionTreeClassifier(criterion='entropy', splitter='random', max_depth=13)
result_tree.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=13,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='random')

In [26]:
y_pred_tr =result_tree.predict(x_test)
print('Accuracy: %.2f' % metrics.accuracy_score(y_test, y_pred_tr))

Accuracy: 0.75


In [30]:
print(metrics.classification_report(y_test, y_pred_tr))


              precision    recall  f1-score   support

   -100874.0       0.00      0.00      0.00         1
   -100702.0       0.79      0.70      0.75        27
     -1004.0       0.75      1.00      0.86         3
     -1002.0       0.00      0.00      0.00         2
       -38.0       0.70      0.93      0.80        15
         0.0       0.71      0.71      0.71         7

   micro avg       0.75      0.75      0.75        55
   macro avg       0.49      0.56      0.52        55
weighted avg       0.71      0.75      0.72        55



/home/centos/anaconda3/envs/dev/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:

from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

In [43]:
clf = RandomForestClassifier(n_estimators=100, max_depth=11,
                             random_state=0)
clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
print(metrics.classification_report(y_test, y_pred))
print('Accuracy: %.2f' % metrics.accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

   -100874.0       0.00      0.00      0.00         1
   -100702.0       0.80      0.89      0.84        27
     -1004.0       1.00      1.00      1.00         3
     -1002.0       0.00      0.00      0.00         2
       -38.0       0.81      0.87      0.84        15
         0.0       0.83      0.71      0.77         7

   micro avg       0.82      0.82      0.82        55
   macro avg       0.57      0.58      0.58        55
weighted avg       0.77      0.82      0.79        55

Accuracy: 0.82


/home/centos/anaconda3/envs/dev/lib/python3.5/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/home/centos/anaconda3/envs/dev/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
